# Brute Force with Numpy

In [2]:
import json
import numpy as np
import pandas as pd

from joblib import Parallel, delayed
from tqdm.notebook import tqdm

## Load Data

In [3]:
with open('data/wordle-candidates.json', 'r') as file:
    wordle_candidates = json.load(file)
    
with open('data/wordle-answers.json', 'r') as file:
    wordle_answers = json.load(file)

wordle_candidates = pd.DataFrame(wordle_candidates['words'], columns=['word'])
wordle_answers = pd.DataFrame(wordle_answers['words'], columns=['word'])
wordle_candidates['is_answer'] = 0
wordle_answers['is_answer'] = 1
wordle = wordle_candidates.append(wordle_answers).reset_index(drop=True)

In [4]:
words_all = pd.read_table('data/archive/en_words_1_5-5.txt', delimiter=' ', header=None, index_col=None,
                         names=['word_len', 'word_freq', 'n_articles']).reset_index()
words_all = words_all.rename(columns={'index': 'word'})

# Filter by english
alphabet = list('abcdefghijklmnopqrstuvwxyz')
words_all = words_all.loc[words_all.word.apply(lambda x: all([l in alphabet for l in x]))].reset_index(drop=True)

## Prepare Artifacts

In [5]:
alpha_dict = {l: i for i, l in enumerate(list('abcdefghijklmnopqrstuvwxyz'))}

In [6]:
# Initialise solutions vector
solutions = np.zeros((wordle_answers.shape[0], 26, 5), dtype='int8')
for i, word in enumerate(wordle_answers.word):
    for j, l in enumerate(word):
        solutions[i, alpha_dict[l], j] = 1
        
        
# Initialise candidates vector
candidates = np.zeros((wordle.shape[0], 26, 5), dtype='int8')
for i, word in enumerate(wordle.word):
    for j, l in enumerate(word):
        candidates[i, alpha_dict[l], j] = 1

## Game Logic

In [7]:
def get_feedback(input_word, solution):
    output = ''
    for i in range(5):
        if input_word[i] == solution[i]:
            output += 'G'
        elif input_word[i] in solution:
            output += 'Y'
        else:
            output += 'X'
    return output

In [8]:
def filter_wordset(input_word, feedback, wordset):
    newset = wordset.copy()
    for i in range(5):
        if feedback[i] == 'G':
            newset = newset.loc[newset.word.str[i] == input_word[i]]
        elif feedback[i] == 'Y':
            # newset = newset.loc[newset.word.str.contains(input_word[i])]
            newset = newset.loc[newset.word.str.contains(input_word[i]) & newset.word.apply(lambda x: x[i] != input_word[i])]
        else:
            newset = newset.loc[~newset.word.str.contains(input_word[i])]
    return newset

## Vector Ops

In [9]:
def init_vec(word):
    mat = np.zeros((26, 5), dtype='int8')
    for i, l in enumerate(word):
        mat[alpha_dict[l], i] = 1
    return mat

In [10]:
def get_scores(word, mask):
    word_vec = init_vec(word)
    solutions_masked = solutions[mask]
    greens = solutions_masked * word_vec
    yellows = word_vec * (
        (solutions_masked.sum(axis=2) >= word_vec.sum(axis=1)) & 
        (word_vec.sum(axis=1) > 0)) \
        .reshape(np.sum(mask), 26, 1) - greens
    greys = word_vec - greens - yellows
    scores = np.array([np.sum(greens, axis=(1,2)), np.sum(yellows, axis=(1,2)), np.sum(greys, axis=(1,2))]).T
    # scores = []
    # for i in np.array(range(solutions.shape[0]))[mask]:
    #     solution = solutions[i]
    #     greens = solution * word_vec
    #     yellows = word_vec * ((solution.sum(axis=1) >= word_vec.sum(axis=1)) & (word_vec.sum(axis=1) > 0)).reshape(26, 1) - greens
    #     greys = word_vec - greens - yellows
    #     scores.append((np.sum(greens), np.sum(yellows), np.sum(greys)))
        
    return scores

In [11]:
def get_final_scores(word, mask):
    scores = get_scores(word, mask)
    df_scores = pd.DataFrame(scores, columns=['g', 'y', 'x'])
    df_scores['score'] = df_scores.g * 2 + df_scores.y
    
    return df_scores.score.mean()

In [12]:
def get_n_cands(word, filter_mask, candidate_mask):
    return np.sum((np.sum((filter_mask * solutions[candidate_mask]) == solutions[candidate_mask], axis=(-2,-1)) == 130))

In [16]:
get_scores('happy', [True] * 2315)

array([[0, 1, 4],
       [0, 0, 5],
       [1, 0, 4],
       ...,
       [1, 1, 3],
       [0, 1, 4],
       [0, 2, 3]])

### Filter Mask

In [12]:
def init_candidate_mask():
    return np.array([True] * wordle_answers.shape[0])

In [13]:
def init_filter_mask():
    filter_mask = np.ones((26,5))
    return filter_mask

In [14]:
def update_filter_mask(input_word, feedback, mask):
    wv = init_vec(input_word)
    row_idx = [alpha_dict[l] for l in input_word]
    output = mask.copy()
    for i, (fb, r) in enumerate(zip(feedback, row_idx)):
        # Green
        if fb == 'G':
            output[:, i] = 0
            output[r, i] = 1
        # Yellow
        elif fb == 'Y':
            output[r, i] = 0
        # Grey
        elif fb == 'X':
            output[r, :] = 0
    return output

In [15]:
def update_candidate_mask(input_word, feedback, wordset, mask):
    newmask = mask.copy()
    for i in range(5):
        if feedback[i] == 'G':
            newmask[~wordset.word.str[i].eq(input_word[i])] = False
        elif feedback[i] == 'Y':
            newmask[~(wordset.word.str.contains(input_word[i]) & wordset.word.apply(lambda x: x[i] != input_word[i]))] = False
        elif feedback[i] == 'X':
            newmask[wordset.word.str.contains(input_word[i])] = False
            
    return newmask

## Global Scores

In [16]:
def compute_letter_frequencies(wordset):
    w = wordset.copy()
    for letter in list('abcdefghijklmnopqrstuvwxyz'):
        w[letter] = w.word.str.contains(letter).astype(int)
    return w.iloc[:, 1:]

def compute_score(x, freqs):
    letters = set(x)
    output = 0
    for letter in letters:
        output += freqs[letter]
    return output

In [17]:
global_freqs = compute_letter_frequencies(wordle).sum().to_dict()
global_scores = wordle.word.apply(compute_score, freqs=global_freqs)
global_scores = pd.DataFrame({'word': wordle.word, 'score': global_scores})
global_scores = global_scores.merge(words_all[['word', 'word_freq', 'n_articles']], how='left', left_on='word', right_on='word')
global_scores = global_scores.fillna(0).sort_values(['score', 'word_freq'], ascending=False)

In [120]:
# Function to compute number of candidates based on GYX
def compute_cands(gyx_triplet):
    
    gyx_triplet = gyx_triplet.reshape(3,26,5)
    
    # Green checks
    if np.sum(gyx_triplet[0]) > 0:
        green_boolean = np.sum(gyx_triplet[0] * solutions == gyx_triplet[0], axis=(-2,-1)) == 130
        filtered_solutions = solutions[green_boolean]
    else:
        filtered_solutions = solutions.copy()
    
    # Yellow avoid: All yellow locations are zero
    if np.sum(gyx_triplet[1]) > 0:
        yellow_avoid = np.sum(gyx_triplet[1] * filtered_solutions == 0, axis=(-2,-1)) == 130

        # Yellow present: 
        # 1. Compute row sums for yellow vector
        # 2. Select rows with at least one yellow in each solution word vector
        # 3. Compute row sums for solution vector to check there are at least one
        # 4. Check that there are two
        yellow_sums = np.sum(gyx_triplet[1], axis=-1)
        yellow_present = np.sum(
            np.sum(filtered_solutions[:, yellow_sums >= 1, :], axis=-1) >= 1,
            axis=-1) == 2

        # Combine yellow checks
        yellow_boolean = yellow_present * yellow_avoid

        # Filter based on yellows
        filtered_solutions = filtered_solutions[yellow_boolean]

    # Grey checks
    if np.sum(gyx_triplet[2]) > 0:
        grey_boolean = np.sum(np.sum(gyx_triplet[2], axis=-1, keepdims=True) * filtered_solutions == 0, axis=(-2,-1)) == 130
        filtered_solutions = filtered_solutions[grey_boolean]

    # Count no. of candidates
    return filtered_solutions.shape[0]

In [121]:
def get_ncands(word):
    # Initialise candidate word vector
    word_vec = init_vec(word)

    # Compute greens, yellows, and greys
    greens = candidates * word_vec
    yellows = word_vec * (
        (candidates.sum(axis=2) >= word_vec.sum(axis=1)) & 
        (word_vec.sum(axis=1) > 0)) \
        .reshape(12972, 26, 1) - greens
    greys = word_vec - greens - yellows

    # Set up GYX tensor
    gyx_reshaped = np.stack([greens, yellows, greys], axis=1)
    gyx_reshaped = gyx_reshaped.reshape(12972, 390)
    
    # Compute raw candidate data
    ncands = np.apply_along_axis(compute_cands, 1, gyx_reshaped)
    ncands_max = np.max(ncands)
    ncands_mean = np.mean(ncands)
    
    return word, ncands_max, ncands_mean

In [122]:
import time

In [124]:
t0 = time.time()
z1 = get_ncands('anana')
print(time.time() - t0)

14.319398403167725


In [125]:
z1

('anana', 1054, 785.949892075239)

## Tests

In [129]:
test1 = Parallel(n_jobs=5, verbose=1)(delayed(get_ncands)(word) for word in global_scores.word.iloc[:500])

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.4min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:  6.1min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 14.2min
[Parallel(n_jobs=5)]: Done 500 out of 500 | elapsed: 16.1min finished


In [132]:
df1 = pd.DataFrame(test1, columns=['word', 'ncands_max', 'ncands_mean'])
df1.sort_values('ncands_max').head(10)

,word,ncands_max,ncands_mean
461,sauce,243,40.464770
245,noise,254,44.387835
319,sayne,256,38.804502
141,saute,276,38.646315
137,saice,278,36.816220
197,toise,280,43.883287
129,salue,284,40.062673
42,saine,288,36.460762
460,cause,300,46.460299
474,pause,302,50.530065


In [134]:
df1['rank_max'] = df1.ncands_max.rank()
df1['rank_mean'] = df1.ncands_mean.rank()
df1['avg_rank'] = df1[['rank_max', 'rank_mean']].mean(axis=1)

In [135]:
df1.sort_values('avg_rank').head(10)

,word,ncands_max,ncands_mean,rank_max,rank_mean,avg_rank
137,saice,278,36.816220,5.0,3.0,4.00
141,saute,276,38.646315,4.0,4.0,4.00
319,sayne,256,38.804502,3.0,5.0,4.00
42,saine,288,36.460762,8.0,2.0,5.00
461,sauce,243,40.464770,1.0,15.0,8.00
129,salue,284,40.062673,7.0,12.0,9.50
222,claes,321,40.606075,13.0,17.0,15.00
8,aloes,364,34.039624,36.5,1.0,18.75
12,lares,370,40.109235,43.0,13.0,28.00
66,salet,353,41.461918,30.0,29.0,29.50


In [126]:
test2 = Parallel(n_jobs=5, verbose=1)(delayed(get_ncands)(word) for word in wordle.word.iloc[:100])

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.7min
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:  4.3min finished


In [128]:
df2 = pd.DataFrame(test2, columns=['word', 'ncands_max', 'ncands_mean'])
df2.sort_values('ncands_max').head(50)

,word,ncands_max,ncands_mean
43,abune,344,66.645698
30,aboil,348,63.762026
54,aceta,359,114.152559
75,actin,378,99.115171
59,acids,383,64.878122
84,addio,390,82.710762
56,ached,390,91.699198
88,adieu,394,48.431545
70,acold,400,96.866019
98,adred,409,110.192800


## App

In [18]:
def run_app(pre_load=None):
    candidate_mask = init_candidate_mask()
    filter_mask = init_filter_mask()
    step = 1
    w = wordle.copy()
    res = pd.DataFrame([{'a': 1}, {'a': 1}])
    tested_words = []
    all_chars = []
    
    while res.shape[0] > 1:
        print(f'[ ---- STEP {step} ----]')
        if not (step == 1 and pre_load):
            guess = input('Input a guess:')
        else:
            guess = pre_load
        if guess.lower() in ['quit', 'q']:
            return w, candidate_mask, res
        tested_words.append(guess)
        all_chars = all_chars + list(set(guess))
        all_chars = list(set(all_chars))
        
        fb = input('Input feedback:')
        if fb.lower() in ['quit', 'q']:
            return w, candidate_mask, res
        
        # Update masks and candidate set
        candidate_mask = update_candidate_mask(guess, fb.upper(), wordle, candidate_mask)
        filter_mask = update_filter_mask(guess, fb, filter_mask)
        w = filter_wordset(guess, fb.upper(), w)
        
        # Candidates
        print(f'Found {w.shape[0]} candidates. Running analysis...')
        new_scores = []
        new_ncands = []
        
        # eval_mat = solutions[candidate_mask].copy()
        
        for word in tqdm(w.word):
            # Get G/Y/X scores
            new_scores.append(get_final_scores(word, candidate_mask))
            
            # Get projected filter mask
#             temp_cands = 0
#             for solution in w.word:
#                 proj_fb = get_feedback(word, solution)
#                 proj_filter_mask = update_filter_mask(word, proj_fb, filter_mask)
#                 temp_cands += get_n_cands(word, proj_filter_mask, candidate_mask)
            
#             new_ncands.append(np.mean(temp_cands / w.shape[0]))
            
        
        print(f'Suggestions for step {step + 1} ({w.shape[0]}):')
        res = pd.DataFrame({'word': w.word, 'score': new_scores}) \
            .merge(words_all[['word', 'word_freq']], on='word', how='left') \
            .fillna(0)
        # display(res.sort_values(['ncands', 'score', 'word_freq'], ascending=[True, False, False]).head(10))
        display(res.sort_values(['score', 'word_freq'], ascending=False).head(10))
        
        # Filters
        # if w.shape[0] > 10:
        #     print(f'\nLarge number of candidates found ({w.shape[0]}). We recommend filtering the candidates more:')
        #     display(
        #         global_scores.loc[global_scores.word.apply(lambda x: len(set(all_chars).intersection(set(list(x)))) < 1)] \
        #             .head(5)
        #     )
        if w.shape[0] <= 10:
            print(f'Small number of candidates remaining ({w.shape[0]}). We recommend choosing the most popular option:')
            display(res.sort_values(['word_freq', 'score'], ascending=False).head(5))
        
        # Check for repeats
        if w.shape[0] <= 8 and w.shape[0] >= 3:
            w_copy = res.sort_values(['word_freq', 'score'], ascending=False).copy()
            # Extract letters
            for i in range(5):
                w_copy[f'p{i}'] = w_copy.word.str[i]
            
            # Count the number of unique columns
            unique_mask = w_copy.iloc[:, -5:].nunique() > 1
            
            # If only 1, then recommend another word
            if unique_mask.sum() == 1:
                wc = wordle.copy()
                total_letters = w_copy.shape[0]
                wc['scores'] = 0
                wc['counts'] = 0
                for i, letter in enumerate(np.squeeze(w_copy[unique_mask.index[unique_mask]].values)):
                    wc['scores'] = wc['scores'] + (total_letters - i) * wc.word.str.contains(letter).astype(int)
                    wc['counts'] = wc['counts'] + wc.word.str.contains(letter).astype(int)
                    
                print(f'\nWords with only one letter differential detected. Consider filtering:')
                display(wc.loc[wc.counts.le(total_letters // 2 * 3)].sort_values('scores', ascending=False).head(5))

                
        step += 1
        print()
        
    return w, candidate_mask, res

## Run

In [25]:
curr_wordset, curr_mask, curr_res = run_app('soare')

[ ---- STEP 1 ----]


Input feedback: xxxyy


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2315 but corresponding boolean dimension is 12972